# Chapter 5. 순환 신경망(RNN)

지금까지 살펴본 신경망은 `피드포워드(feed forward)` 라는 유형의 신경망  
- `피드 포워드` : 흐름이 단방향인 신경망
  - 장점 : 구성이 단순하여 구조를 이해하기 쉽고 많은 문제에 응용 가능
  - 단점 : 시계열 데이터를 잘 다루지 못함

#### 이번 단원의 목표 : `피드포워드`의 문제점을 지적하고, `RNN`의 구조를 이해해 그 문제점을 해결

## 5.1 확률과 언어 모델

### 5.1.1 `word2vec`을 확률 관점에서 바라보다

#### `word2vec` 의 `CBOW` 모델

$ w_1, w_2, ..., w_{T} $ 라는 단어열로 표현되는 말뭉치을 생각

$w_{t-1}$과 $w_{t+1}$이 주어졌을 때 타깃이 $w_{t}$가 될 확률 (윈도우 크기가 1일 때)

$$ P(w_t | w_{t-1}, w_{t+1})$$

맥락을 좌우 대칭이 아니라 왼쪽 윈도우만으로 한정해서 생각해보자.  
<img src="./master/images/fig 5-2.png" width=600/>

$$ P(w_t | w_{t-2}, w_{t-1})$$

`교차 엔트로피 오차`를 이용해서 구한 위 식의 손실 함수  
$$L = -\log P(w_t | w_{t-1}, w_{t+1})$$

### 5.1.2 언어 모델

- `언어 모델` : 단어 나열에 확률을 부여. 특정한 단어의 시퀀스에 대해서, 그 시퀀스가 얼마나 자연스러운지 확률로 평가
- `언어 모델`은 새로운 문장을 생성하는 용도로도 이용할 수 있다.
- `CBOW` 모델은 단어 순서를 무시하기 때문에 `언어 모델`로 적합하지 않다.
- `RNN(순환신경망)`은 긴 시계열 데이터에도 대응할 수 있다.

## 5.2 RNN이란

### 5.2.1 순환하는 신경망

순환하기 위해서는 `닫힌 경로`가 필요

<p align = 'left'><img src="./master/images/fig 5-6.png" height=250 /></p>  
  
- $x_{t}$ : 각 단어의 분산 표현 (단어 벡터)

### 5.2.2 순환 구조 펼치기

<p align = 'left'><img src="./master/images/fig 5-8.png" height=250 /></p>  


<p align = 'left'><img src="./master/images/e 5-9.png" /></p>  

### 5.2.3 BPTT

<p align = 'left'><img src="./master/images/fig 5-10.png" height=200 /></p>  


`RNN`에서의 오차역전파법은 '시간 방향으로 펼친 신경망의 오차역전파법'이란 뜻으로 `BPTT(Backpropagation Through Time)`라고 한다.

시계열 데이터의 시간 크기가 커지는 것에 비례하여 `BPTT`가 소비하는 컴퓨팅 자원도 증가한다는 단점이 있다.

### 5.2.4 Truncated BPTT

`BPTT`의 문제를 해결하기 위해, 신경망을 적당한 지점에서 잘라서 작은 신경망 여러 개로 만드는 `Truncated BPTT`를 사용한다.

<p align = 'left'><img src="./master/images/fig 5-14.png" height=300 /></p>  


### 5.2.5 Truncated BPTT의 미니배치 학습

<p align = 'left'><img src="./master/images/fig 5-15.png" height=500 /></p>  


- 길이가 1000인 시계열 데이터
- 시각의 길이를 10개로 자르는 `Truncated BPTT` 로 학습
- 미니배치의 수 : 2개

## 5.3 RNN 구현

<p align = 'left'><img src="./master/images/fig 5-17.png" height=300 /></p>  


- `Time RNN` 계층 내에서 한 단계의 작업을 수행하는 계층 : `RNN` 계층
- $T$ 개 단계분의 작업을 한꺼번에 처리하는 계층 : `Time RNN` 계층

### 5.3.1 RNN 계층 구현

<p align = 'left'><img src="./master/images/fig 5-18.png" height=200 /></p>  
  
#### forward 구현


In [2]:
import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next

<p align = 'left'><img src="./master/images/fig 5-20.png" height=300 /></p>  
  
#### backward 구현

In [3]:

import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.t)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev

### 5.3.2 Time RNN 계층 구현

In [7]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False): # stateful : 은닉 상태를 인계받을지
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        
        self.h, self.dh = None
        self.stateful = stateful
        
    def set_state(self, h):
        self.h = h
        
    def reset_state(self):
        self.h = None
        
#forward()#####################################################################################
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape
        
        self.layers = []
        hs = np.empty((N, T, H), dtype='f')
        
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
            
        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)
            
        return hs

#backward()######################################################################################
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape
        
        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh) # 합산된 기울기 (?)
            dxs[:, t, :] = dx
            
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
                
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        
        return dxs

## 5.4 시계열 데이터 처리 계층 구현

`RNNLM` - RNN Language Model

### 5.4.1 RNNLM의 전체 그림

<p align = 'left'><img src="./master/images/fig 5-25.png" height=400 /></p>  


1. `Embedding` : 단어 ID를 단어의 분산 표현(단어 벡터)으로 변환합니다.
2. `RNN` : 은닉 상태를 다음 층으로 출력함과 동시에, 다음 시간의 RNN 계층으로 출력합니다.
3. `Affine` : RNN 계층이 위로 출력한 은닉 상태는 Affine 계층을 거칩니다. 
4. `Softmax` : 마지막 출력

`RNNLM` 모델은 **특정 단어 다음에 어떤 단어가 올지 예측**한다.

### 5.4.2 Time 계층 구현

#### Time Embedding

In [10]:
from common.layers import *

class TimeEmbedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W
        
    def forward(self, xs):
        N, T = xs.shape
        V, D = self.W.shape
        
        out = np.empty((N, T, D), dtype='f')
        self.layers = []
        
        for t in range(T):
            layer = Embedding(self.W)
            out[:, t, :] = layer.forward(xs[:, t])
            self.layers.append(layer)
            
        return out
    
    def backward(self, dout):
        N, T, D = dout.shape
        
        grad = 0
        for t in range(T):
            layer = self.layers[t]
            layer.backward(dout[:, t, :])
            grad += layer.grad[0]
            
        self.grads[0][...] = grad
        return None

## 5.5 RNNLM 학습과 평가

### 5.5.1 RNNLM 구현

<p align = 'left'><img src="./master/images/fig 5-30.png" width=400 /></p>  


In [3]:
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import *

class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        # 가중치 초기화
        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]
        
        # 모든 가중치의 기울기를 리스트에 모은다.
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        self.rnn_layer.reset_state() # TODO : reset_state 가 뭔데?
        
        

### 5.5.2 언어 모델의 평가

`perplexity` 를 사용해서 언어 모델을 평가한다.

### 5.5.3 RNNLM의 학습 코드

In [4]:
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
from common.optimizer import SGD
from dataset import ptb
# from simple_rnnlm import SimpleRnnlm

# 하이퍼파라미터 설정
batch_size = 10
wordvec_size = 100
hidden_size = 100 # RNN의 은닉 상태 벡터의 원소 수
time_size = 5 # Truncated BPTT가 한 번에 펼치는 시간 크기
lr = 0.1
max_epoch = 100

# 학습 데이터 읽기 (전체 중 1000개만)
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1) # TODO

xs = corpus[:-1] # 입력 TODO
ts = corpus[1:] # 출력(정답 레이블) TODO
data_size = len(xs)
print("말뭉치 크기: %d, 어휘 수: %d" % (corpus_size, vocab_size))

# 학습 시 사용하는 변수
max_iters = data_size // (batch_size * time_size)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

# (1) 각 미니배치에서 샘플을 읽기 시작 위치를 계산
jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)]

for epoch in range(max_epoch):
    for iter in range(max_iters):
        # (2) 미니배치 획득
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1
            
        # 기울기를 구하여 매개변수 갱신
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1
        
    # (3) Epoch 마다 perplexity 평가
    ppl = np.exp(total_loss / loss_count)
    print("| 에폭 %d | 퍼플렉서티 %.2f"
          % (epoch+1, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0

말뭉치 크기: 1000, 어휘 수: 418
| 에폭 1 | 퍼플렉서티 407.24
| 에폭 2 | 퍼플렉서티 288.55
| 에폭 3 | 퍼플렉서티 225.03
| 에폭 4 | 퍼플렉서티 214.61
| 에폭 5 | 퍼플렉서티 205.26
| 에폭 6 | 퍼플렉서티 203.18
| 에폭 7 | 퍼플렉서티 198.96
| 에폭 8 | 퍼플렉서티 196.43
| 에폭 9 | 퍼플렉서티 191.38
| 에폭 10 | 퍼플렉서티 192.82
| 에폭 11 | 퍼플렉서티 188.47
| 에폭 12 | 퍼플렉서티 191.69
| 에폭 13 | 퍼플렉서티 188.72
| 에폭 14 | 퍼플렉서티 189.81
| 에폭 15 | 퍼플렉서티 188.98
| 에폭 16 | 퍼플렉서티 185.43
| 에폭 17 | 퍼플렉서티 182.72
| 에폭 18 | 퍼플렉서티 180.46
| 에폭 19 | 퍼플렉서티 181.28
| 에폭 20 | 퍼플렉서티 183.78
| 에폭 21 | 퍼플렉서티 181.01
| 에폭 22 | 퍼플렉서티 177.81
| 에폭 23 | 퍼플렉서티 172.74
| 에폭 24 | 퍼플렉서티 175.96
| 에폭 25 | 퍼플렉서티 170.65
| 에폭 26 | 퍼플렉서티 170.22
| 에폭 27 | 퍼플렉서티 167.14
| 에폭 28 | 퍼플렉서티 163.70
| 에폭 29 | 퍼플렉서티 161.65
| 에폭 30 | 퍼플렉서티 155.49
| 에폭 31 | 퍼플렉서티 154.55
| 에폭 32 | 퍼플렉서티 150.53
| 에폭 33 | 퍼플렉서티 148.93
| 에폭 34 | 퍼플렉서티 143.75
| 에폭 35 | 퍼플렉서티 142.63
| 에폭 36 | 퍼플렉서티 136.57
| 에폭 37 | 퍼플렉서티 131.05
| 에폭 38 | 퍼플렉서티 127.72
| 에폭 39 | 퍼플렉서티 122.37
| 에폭 40 | 퍼플렉서티 116.11
| 에폭 41 | 퍼플렉서티 117.88
| 에폭 42 | 퍼플렉서티 111.70
| 에폭 43 | 퍼플렉서티 104

### 5.5.4 RNNLM의 Trainer 클래스

`ch05/train.py` 참고